In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

In [2]:
feature_number = 105

In [3]:
X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [4]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.fit_transform(X)

In [5]:
pca = PCA(n_components= 0.999).fit(X_scaled)
X_pca = pca.transform(X_scaled)
print(X_pca.shape)

(393345, 82)


In [6]:
model_X = X_pca
model_y = y
print(model_X.shape, model_y.shape)

(393345, 82) (393345,)


In [7]:
# find how many kinds of values in y
y_values = np.unique(model_y)
print(y_values)

[0. 1. 2. 3. 4. 5. 6.]


In [8]:
from cnn_builder2 import *


# learning_rate = 2e-2
# channel = 32


# model_X_normalized = torch.Tensor(model_X)
# model_X_normalized = torch.unsqueeze(model_X_normalized, 1)


# cnn = CNN_Sleeping(channels = channel)

# model = skorch.NeuralNetClassifier(cnn, criterion=torch.nn.CrossEntropyLoss,
#                              device="cuda",
#                              optimizer=torch.optim.SGD ,
#                              lr=learning_rate,
#                              max_epochs=120,
#                              batch_size=128,
#                              callbacks=[skorch.callbacks.EarlyStopping(lower_is_better=True)])

# model.fit(model_X_normalized, np.asarray(model_y, dtype=np.int64))

train_cnn_model(model_X, model_y)

Start with learning rate: 0.01 and channel: 32
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.9684       0.7155        0.9404  5.3925
      2        0.8287       0.8350        0.5457  5.1442
      3        0.4886       0.8590        0.4477  5.0885
      4        0.4194       0.8743        0.3931  4.9244
      5        0.3750       0.8819        0.3561  4.9147
      6        0.3478       0.8868        0.3345  5.0698
      7        0.3301       0.8915        0.3185  4.9883
      8        0.3167       0.8953        0.3061  4.9331
      9        0.3061       0.8979        0.2967  4.9035
     10        0.2975       0.9004        0.2895  4.8470
     11        0.2906       0.9019        0.2839  4.8932
     12        0.2852       0.9030        0.2793  4.9070
     13        0.2807       0.9039        0.2754  4.9630
     14        0.2769       0.9049        0.2722  5.0205
     15        0.2735       0.9056       

In [9]:
# distribution of labels y
# print every count of labels
for i in y_values:
    print(i, np.sum(model_y == i))


0.0 281216
1.0 15578
2.0 62754
3.0 5255
4.0 3231
5.0 24275
6.0 1036
